In [ ]:
!pip uninstall -y torch torchvision torchaudio numpy
!pip install numpy==1.26.4
!pip install torch==2.2.0 torchvision==0.17.0
!pip install transformers==4.39.3 bitsandbytes==0.45.2 peft==0.10.0 accelerate==0.27.2 datasets sentencepiece rouge-score evaluate


Found existing installation: torch 2.2.0
Uninstalling torch-2.2.0:
  Successfully uninstalled torch-2.2.0
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; pytho

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 118.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, which is not installed.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.4 MB/s eta 0:00:00
  Using cached bitsandbytes-0.45.2-py3-none-manylinux_2_24_x86_64.whl.metadata (5.8 kB)
  Using cached accelerate-0.27.2-py3-none-any.whl.metadata (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 103.1 MB/s eta 0:00:00
Using cached bitsandbytes-0.45.2-py3-none-manylinux_2_24_x86_64.whl (69.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 18.5 MB/s eta 0:00:00
Using cached accelerate-0.27.2-py3-none-any.whl (279 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.39.3 which is incompatible.


In [ ]:
import numpy, torch, bitsandbytes, accelerate
print("NumPy:", numpy.__version__)
print("Torch:", torch.__version__)
print("BitsAndBytes:", bitsandbytes.__version__)
print("Accelerate:", accelerate.__version__)


AttributeError: module 'torch' has no attribute 'uint64'

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    Seq2SeqTrainer, Seq2SeqTrainingArguments,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, TaskType
import torch

# Model config
model_name = "google/flan-t5-large"
MAX_INPUT_LEN = 1024
MAX_TARGET_LEN = 1024

# Output dir
output_dir = "/content/drive/MyDrive/AIScientist/FlanT5/flan_t5large_lora_chunked"


RuntimeError: Failed to import transformers.trainer_seq2seq because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
module 'torch' has no attribute 'uint64'

In [ ]:
dataset = load_dataset(
    "json",
    data_files={
        "train": "/content/drive/MyDrive/AIScientist/FlanT5/train_ready_trunc.json",
        "test": "/content/drive/MyDrive/AIScientist/FlanT5/test_ready_trunc.json"
    }
)

# 87.5% train, 12.5% val → 70/10 split overall
split_dataset = dataset["train"].train_test_split(test_size=0.125, seed=42)
train_data = split_dataset["train"]
val_data = split_dataset["test"]
test_data = dataset["test"]


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def chunk_tokens(token_ids, max_len):
    return [token_ids[i:i+max_len] for i in range(0, len(token_ids), max_len)]

def preprocess_chunked_function(examples):
    chunked_inputs, chunked_labels, attention_masks = [], [], []

    for inp_text, tgt_text in zip(examples["input"], examples["output"]):
        # Tokenize without truncation
        inp_ids = tokenizer(inp_text, truncation=False)["input_ids"]
        tgt_ids = tokenizer(tgt_text, truncation=False)["input_ids"]

        # Split into chunks
        input_chunks = chunk_tokens(inp_ids, MAX_INPUT_LEN)
        target_chunks = chunk_tokens(tgt_ids, MAX_TARGET_LEN)

        # Pair chunks and append
        for ic, tc in zip(input_chunks, target_chunks):
            chunked_inputs.append(ic)
            chunked_labels.append(tc)
            attention_masks.append([1] * len(ic))

    # Debug: print number of chunks (first call only)
    if len(chunked_inputs) > 0:
        print(f"Created {len(chunked_inputs)} chunks for this batch.")

    return {
        "input_ids": chunked_inputs,
        "attention_mask": attention_masks,
        "labels": chunked_labels
    }

train_tokenized = train_data.map(
    preprocess_chunked_function,
    batched=True,
    remove_columns=train_data.column_names
)
val_tokenized = val_data.map(
    preprocess_chunked_function,
    batched=True,
    remove_columns=val_data.column_names
)
test_tokenized = test_data.map(
    preprocess_chunked_function,
    batched=True,
    remove_columns=test_data.column_names
)


In [ ]:
!nvidia-smi


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)



In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=32,            # higher capacity
    lora_alpha=64,   # proportional
    lora_dropout=0.1,
    bias="none"
)

model = get_peft_model(model, peft_config)

model.config.use_cache = False
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

model.is_loaded_in_4bit = True
model.is_loaded_in_8bit = False

# Freeze base params, train LoRA only
for param in model.parameters():
    param.requires_grad = False
for name, param in model.named_parameters():
    if "lora" in name:
        param.requires_grad = True


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    logging_steps=100,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=2,
    predict_with_generate=True,
    generation_max_length=MAX_TARGET_LEN,
    warmup_steps=500,
    ddp_find_unused_parameters=False,
    report_to="none"
)


In [ ]:
from evaluate import load

rouge = load("rouge")
bleu = load("bleu")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    rouge_res = rouge.compute(predictions=preds, references=labels)
    bleu_res = bleu.compute(predictions=preds, references=[[l] for l in labels])
    return {
        "rouge1": round(rouge_res["rouge1"] * 100, 2),
        "rouge2": round(rouge_res["rouge2"] * 100, 2),
        "rougeL": round(rouge_res["rougeL"] * 100, 2),
        "bleu": round(bleu_res["bleu"] * 100, 2)
    }


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


In [ ]:
import torch
torch.cuda.empty_cache()

trainer.train()


In [ ]:
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


In [ ]:
def evaluate_model(model, tokenizer, dataset, max_target_len=1024):
    predictions = []
    references = []

    for example in dataset:
        inputs = tokenizer(example["input"], return_tensors="pt", truncation=False).to("cuda")

        # Generate in chunks
        output_ids = model.generate(**inputs, max_length=max_target_len)
        pred_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        predictions.append(pred_text)
        references.append(example["output"])

    rouge_res = rouge.compute(predictions=predictions, references=references)
    bleu_res = bleu.compute(predictions=predictions, references=[[ref] for ref in references])

    return rouge_res, bleu_res, predictions, references

torch.cuda.empty_cache()
rouge_res, bleu_res, preds, refs = evaluate_model(model, tokenizer, test_data)

print("=== ROUGE ===")
print(rouge_res)
print("\n=== BLEU ===")
print(bleu_res)


In [ ]:
for i in range(5):
    print(f"\n--- Example {i+1} ---")
    print(f"Input:\n{test_data[i]['input']}")
    print(f"\nGenerated Output:\n{preds[i]}")
    print(f"\nReference Output:\n{refs[i]}")
